In [1]:
import pandas as pd
df=pd.read_csv("/Users/sohail21400/Desktop/trading_machine/data/EURUSD/EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df

,Gmt time,Open,High,Low,Close,Volume
0,30.09.2019 00:00:00.000,1.09425,1.09426,1.09405,1.09406,585.10
1,30.09.2019 00:05:00.000,1.09408,1.09414,1.09401,1.09409,289.39
2,30.09.2019 00:10:00.000,1.09410,1.09423,1.09408,1.09410,276.24
3,30.09.2019 00:15:00.000,1.09409,1.09410,1.09388,1.09389,439.29
4,30.09.2019 00:20:00.000,1.09390,1.09395,1.09388,1.09395,341.23
...,...,...,...,...,...,...
225081,30.09.2022 20:35:00.000,0.98028,0.98034,0.98001,0.98022,624.12
225082,30.09.2022 20:40:00.000,0.98023,0.98047,0.98007,0.98030,408.20
225083,30.09.2022 20:45:00.000,0.98026,0.98034,0.98019,0.98031,317.29
225084,30.09.2022 20:50:00.000,0.98031,0.98067,0.97987,0.97999,1472.13


In [2]:
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%d.%m.%Y %H:%M:%S')
df.set_index("Gmt time", inplace=True)
df=df[df.High!=df.Low]
len(df)

/var/folders/9j/zg2w0vvd4kd1c9dmf275gb2h0000gn/T/ipykernel_43808/286423569.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Gmt time"]=df["Gmt time"].str.replace(".000","")


224989

In [3]:
import pandas_ta as ta
df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
df['RSI']=ta.rsi(df.Close, length=16)
my_bbands = ta.bbands(df.Close, length=14, std=2.0)
df=df.join(my_bbands)

/var/folders/9j/zg2w0vvd4kd1c9dmf275gb2h0000gn/T/ipykernel_43808/3882960619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["VWAP"]=ta.vwap(df.High, df.Low, df.Close, df.Volume)
/var/folders/9j/zg2w0vvd4kd1c9dmf275gb2h0000gn/T/ipykernel_43808/3882960619.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI']=ta.rsi(df.Close, length=16)


In [4]:
VWAPsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df.Open[i], df.Close[i])>=df.VWAP[i]:
            dnt=0
        if min(df.Open[i], df.Close[i])<=df.VWAP[i]:
            upt=0
    if upt==1 and dnt==1:
        VWAPsignal[row]=3
    elif upt==1:
        VWAPsignal[row]=2
    elif dnt==1:
        VWAPsignal[row]=1

df['VWAPSignal'] = VWAPsignal

In [5]:
def TotalSignal(l):
    if (df.VWAPSignal[l]==2
        and df.Close[l]<=df['BBL_14_2.0'][l]
        and df.RSI[l]<45):
            return 2
    if (df.VWAPSignal[l]==1
        and df.Close[l]>=df['BBU_14_2.0'][l]
        and df.RSI[l]>55):
            return 1
    return 0
        
TotSignal = [0]*len(df)
for row in range(backcandles, len(df)): #careful backcandles used previous cell
    TotSignal[row] = TotalSignal(row)
df['TotalSignal'] = TotSignal

In [6]:
df[df.TotalSignal!=0].count()

Open           3735
High           3735
Low            3735
Close          3735
Volume         3735
VWAP           3735
RSI            3735
BBL_14_2.0     3735
BBM_14_2.0     3735
BBU_14_2.0     3735
BBB_14_2.0     3735
BBP_14_2.0     3735
VWAPSignal     3735
TotalSignal    3735
dtype: int64

In [7]:
import numpy as np
def pointposbreak(x):
    if x['TotalSignal']==1:
        return x['High']+1e-4
    elif x['TotalSignal']==2:
        return x['Low']-1e-4
    else:
        return np.nan

df['pointposbreak'] = df.apply(lambda row: pointposbreak(row), axis=1)

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=10400
dfpl = df[st:st+350]
dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.VWAP, 
                           line=dict(color='blue', width=1), 
                           name="VWAP"), 
                go.Scatter(x=dfpl.index, y=dfpl['BBL_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_14_2.0'], 
                           line=dict(color='green', width=1), 
                           name="BBU")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointposbreak'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="Signal")
fig.show()

In [9]:
dfpl = df[:75000].copy()
import pandas_ta as ta
dfpl['ATR']=ta.atr(dfpl.High, dfpl.Low, dfpl.Close, length=7)
#help(ta.atr)
def SIGNAL():
    return dfpl.TotalSignal

In [10]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = 1.2*self.data.ATR[-1]
        TPSLRatio = 1.5

        if len(self.trades)>0:
            if self.trades[-1].is_long and self.data.RSI[-1]>=90:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
                self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/10, commission=0.00)
stat = bt.run()
stat

Start                     2019-09-30 00:00:00
End                       2020-09-30 04:25:00
Duration                    366 days 04:25:00
Exposure Time [%]                    7.553333
Equity Final [$]                   176.857726
Equity Peak [$]                    177.833038
Return [%]                          76.857726
Buy & Hold Return [%]                7.246403
Return (Ann.) [%]                   56.370325
Volatility (Ann.) [%]               25.656941
Sharpe Ratio                         2.197079
Sortino Ratio                        6.208039
Calmar Ratio                         5.573705
Max. Drawdown [%]                  -10.113618
Avg. Drawdown [%]                   -0.705137
Max. Drawdown Duration       38 days 16:45:00
Avg. Drawdown Duration        1 days 17:44:00
# Trades                                  811
Win Rate [%]                        48.335388
Best Trade [%]                        0.42012
Worst Trade [%]                      -0.27659
Avg. Trade [%]                    

In [11]:
bt.plot(show_legend=False)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/backtesting/_plotting.py:122: UserWarning:

Data contains too many candlesticks to plot; downsampling to '1H'. See `Backtest.plot(resample=...)`

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/backtesting/_plotting.py:148: FutureWarning:

Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/backtesting/_plotting.py:148: FutureWarning:

Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=...) instead.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/backtesting/_plotting.py:148: FutureWarning:

Passing method to Int64Index.get_loc is deprecated and will raise in a future version. Use index.get_indexer([item], method=..

Row(id='1475', ...)